### Problem 1

(100 points) Consider the following problem. 

$$
\hbox{min } f =  x_1^2+(x_2-3)^2
$$
$$
\hbox{s.t. } g_1 =  x_2^2-2x_1\leq 0
$$
$$
 g_2= (x_2-1)^2+5x_1-15\leq 0
$$

Implement an SQP algorithm with line search to solve this problem, starting from
${\bf x}_0=(1,1)^T$. Incorporate the QP subproblem. Use BFGS
approximation for the Hessian of the Lagrangian. Use the
merit function and Armijo Line Search to find the step size.

In [16]:
##
# imports

import numpy as np
import scipy.optimize as opt

In [17]:
##
# function defs

def cost_f(x):
    return x[0]**2.0 + (x[1] - 3)**2.0

def g_1(x):
    return x[1]**2.0 - 2.0 * x[0]

def g_2(x):
    return (x[1] - 1)**2.0 + 5.0 * x[0] - 15.0

def g_f(x):
    return [g_1(x), g_2(x)]

def grad_cost_f(x):
    return np.array([2.0 * x[0], 2.0 * x[1] - 6.0])

def grad_g_f(x):
    return np.array([[-2.0, 5.0], [2.0 * x[1], 2.0 * x[1] - 2.0]])

'''
def line_search_phi(x, params):
    #phi = cost(p) - params['a'] * params['t'][0] * np.dot(grad(p), grad(p))
    phi = cost_f(x) - params['a'] * np.dot(params['t'], grad_cost_f(x))
    return phi
    
def line_search(x, params):
    a = params['a']
    i = 0
    while line_search_phi(x, params) < cost_f(x - a * grad_cost_f(x)):
        a = 0.5 * a
        i = i + 1

        if params['debug_verbose']:
            print("line_search_phi: " + str(line_search_phi(x, params)) + ", cost: " + str(cost_f(x - a * grad_cost_f(x))) + ", a: " + str(a))
            print("p: " + str(x) + "grad: " + str(grad_cost_f(x)))

        if(params['max_i'] < i):
            if params['verbose']:
                print("!!Iteration limit reached in line_search, returning a of " + str(a) + "!!")
                print("line_search_phi: " + str(line_search_phi(x, params)) + ", cost: " + str(cost_f(x - a * grad_cost_f(x))) + ", a: " + str(a))
            return a
    return a
'''
def QP(x, lam, h, W):
    max_iter_count = 100
    for i in range(max_iter_count):
        A = grad_g_f(x)
        fx = grad_cost_f(x)
        h = g_f(x)
        big_mat = np.array([[W, np.transpose(A)], [A, np.zeros(size=np.size(A))]])
        s_lam = np.linalg.pinv(big_mat) @ np.array([-fx, -h])
        s = s_lam[0:2]
        lam = s_lam[2:4]
        mu_mag = np.linalg.norm(lam) # mu and lam are interchangeable here, no equality constraints
        if mu_mag > 0:
            return s, lam
        elif mu_mag <= 0: # add more here
            pass
    print("QP hit max iteration count!")
    return [s, lam]

def line_search(s, lam, mu, f, h, g, A):
    return 0.2 # not yet implimented

def BFGS(w_k, alpha_s, x, lam, mu, f, h, g, A):
    hessian_appx = np.eye(size=np.size(x))
    return hessian_appx

def SQP(x_0, lam_0, mu_0, W_0, epi, A):
    div_L = cost_f(x_k) + np.transpose(mu_k) @ grad_g_f(x_k)
    norm_div_L = np.linalg.norm(div_L)
    while epi < norm_div_L:
        (s_k, lam_mu) = QP()
        alpha_k = line_search()
        x_kp1 = x_k + alpha_k * s_k
        W_est_kp1 = BFGS()
        div_L = cost_f(x_kp1) + np.transpose(x_kp1) @ grad_g_f(x_kp1)
        norm_div_L = np.linalg.norm(div_L)
    return x_kp1


In [18]:
##
# checking real answer 
x_0 = np.array([1.0, 1.0])
constraints = opt.NonlinearConstraint(g_f, [-np.inf, -np.inf], [0.0, 0.0])
result = opt.minimize(cost_f, x_0, constraints=constraints)
print(result.message)
print(result.x)

Optimization terminated successfully
[1.06020715 1.45616424]


* DONE
    * Set up basic structure
    * most of QP
    * checked what answer I should be getting.

* TODO 
    * check QP
    * impliment merit function line search
    * impliment BFGS